### 一、vLLM  安装

In [ ]:
pip install vllm -i https://pypi.tuna.tsinghua.edu.cn/simple

### 二、api 形式启动与调用

In [ ]:
python -m vllm.entrypoints.openai.api_server \
    --model /data/LLM/Qwen/Qwen2.5-0.5B-Instruct \ # 模型地址
    --tensor-parallel-size 2 \ # 张量并行的GPU数量（设置为1表示单GPU部署）
    --max-model-len 4096 \ # 模型支持的最大上下文长度（输入+输出的总token数上限）
    --gpu-memory-utilization 0.5 \ # GPU内存利用率限制
    --enforce-eager # 强制使用PyTorch的Eager模式（提升兼容性或调试便利性）

In [ ]:
from openai import OpenAI

# 初始化客户端，指向vLLM服务
client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key=""
)

# 调用聊天接口
response = client.chat.completions.create(
    #需与启动时的model路径一致
    model="/data/LLM/Qwen/Qwen2.5-0.5B-Instruct",  
    messages=[
        {"role": "user", "content": "解释一下什么是大语言模型"}
    ]
)

# 打印结果
print(response.choices[0].message.content)

### 三、离线推理

In [ ]:
from vllm import LLM, SamplingParams

prompts = [
    "你好，你是谁",
    "动物园里有什么？",
    "中国的首都是？",
    "彩虹有几种颜色",
]

sampling_params = SamplingParams(temperature=0.8, top_p=0.95)

# 初始化LLM（整合命令行所有配置参数）
llm = LLM(
    model="/data/LLM/Qwen/Qwen2.5-0.5B-Instruct",
    trust_remote_code=True,          # 对应 --trust-remote-code
    tensor_parallel_size=2,          # 对应 --tensor-parallel-size 1
    max_model_len=4096,              # 对应 --max-model-len 4096
    gpu_memory_utilization=0.4,      # 对应 --gpu-memory-utilization 0.5
    enforce_eager=True               # 对应 --enforce-eager
)

outputs = llm.generate(prompts, sampling_params)

for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")